In [2]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')

print("============================================")
print("/  RfSurveyMap is running.            /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

import geopy
from geopy.distance import geodesic

config = configparser.ConfigParser()
config.read("../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']

  _SLACK_RFSURVEYCHANNEL_NAME = config['analytics']['rfsurveychannelname']
  _USE_DROPBOX   = config['dropbox']['usedropbox']
  _FALLBACK_IMAGE = config['dropbox']['fallbackimage']

  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])

def logger(level, message):
   if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  logger(_LOG_INFO, "Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  logger(_LOG_ERROR, "You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  RfSurveyMap is running.            /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [11]:
#_LOG_LEVEL = _LOG_DEBUG

#Mote types
_MAINS_MOTE_TYPE = 10000
_BATTERY_MOTE_TYPE = 10002

now = datetime.now(pytz.utc)  #tz Aware
logger(_LOG_DEBUG, "Current time is {}".format(now))

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() + timedelta(hours = 4)

_NODE_SENSORTYPE = 40           # the GEOLOCATION type

motequery = "SELECT * FROM kanji_node WHERE deploystate_id=10001 AND (nodetype_id={} OR nodetype_id={})".format(_MAINS_MOTE_TYPE, _BATTERY_MOTE_TYPE)
logger(_LOG_DEBUG, motequery)
df = pd.read_sql(motequery, conn)

logger(_LOG_DEBUG, "number of motes {}".format(len(df.index)))
for ind in df.index:
    nodename = df['name'][ind]
    node_id = df['idnode'][ind]
    #eventquery = "SELECT node.name AS gatewayname,\
    #              location.description AS location, \
    #              * FROM kanji_eventlog event \
    #              JOIN kanji_node node ON (node.name LIKE concat('%',event.gateway,'%')) \
    #              JOIN kanji_location location ON location.idlocation=node.location_id \
    #              JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
    #              JOIN kanji_slackchannel slackchannel ON customer.slacksystemticketchannel_id=slackchannel.idslackchannel \
    #              WHERE timestamp> NOW() - INTERVAL '1 MINUTE' AND node_id={} AND sensortype_id={} ORDER BY timestamp desc;".format(node_id, _NODE_SENSORTYPE)
    eventquery = "SELECT location.description AS location, event.eval AS marcopolo, * FROM kanji_eventlog event \
                  JOIN kanji_node node on node.idnode=event.node_id \
                  JOIN kanji_location location ON location.idlocation=node.location_id \
                  WHERE timestamp> NOW() - INTERVAL '24 HOUR' \
                  AND node_id={} \
                  AND sensortype_id={} ORDER BY timestamp asc".format(node_id, _NODE_SENSORTYPE)
    logger(_LOG_DEBUG, eventquery)
    df2 = pd.read_sql(eventquery, conn)
    if len(df2.index)>0: 
      logger(_LOG_INFO, "latitude, longitude, color, indicator, gateway, distance, snr, rssi")    
      for ind2 in df2.index:
        timestamp = df2['timestamp'][ind2]
        latitude = df2['bval'][ind2]
        longitude = df2['cval'][ind2]
        altitude = df2['dval'][ind2]
        marcopolo = df2['eval'][ind2]
        lorasnr = df2['lorasnr'][ind2]
        rssi = df2['rssi'][ind2]
        gw_name = df2['gateway'][ind2]
        motelocation = df2['location'][ind2]        
        gatewayquery = "SELECT * FROM kanji_node node JOIN kanji_location location ON location.idlocation=node.location_id WHERE name='{}' LIMIT 1".format(gw_name)
        logger(_LOG_DEBUG, gatewayquery)
        df3 = pd.read_sql(gatewayquery, conn)
        if len(df3.index)>0:
          #ogger(_LOG_DEBUG, "gatewayquery ={}".format(gatewayquery))
          gw_latitude = df3['latitude'][0]
          gw_longitude = df3['longitude'][0]
          #logger(_LOG_INFO, "{}\t{}\tstar1\t{}\t{}".format(gw_latitude, gw_longitude, 'black', gw_name))
                 
          nodelocation = (latitude, longitude)
          gatewaylocation = (gw_latitude, gw_longitude)
          loradistance = (float) (geodesic(nodelocation, gatewaylocation).miles * 5280/3)
          if rssi < -120.0:
            color = 'red'
          elif rssi < -100.0:
            color = 'yellow'
          else:
            color = 'green'
          if loradistance>0.0:
            #logger(_LOG_INFO, "{}\t{}\tcircle1\t{}\t{}\t{} dist={:<5.1f}yd snr={}dB, rssi={}dB".format(latitude, longitude, color, ind2, gw_name, loradistance, lorasnr, rssi))
            #logger(_LOG_INFO, "{},{},circle1,{},{},{},{:<5.1f},{},{}".format(latitude, longitude, color, ind2, gw_name, loradistance, lorasnr, rssi)
            logger(_LOG_INFO, "{}\t{}\tcircle1\t{}\t{}\t{}\t{:<5.1f}\t{}\t{}".format(latitude, longitude, color, ind2, gw_name, loradistance, lorasnr, rssi))   
        
logger(_LOG_INFO, "RfSurveyMap Done!") 

latitude, longitude, color, indicator, gateway, distance, snr, rssi
35.3042030334473	-80.5487594604492	circle1	green	0	Peabody	12112.9	-3.8	-93
35.3538513183594	-80.4465179443359	circle1	green	1	Peabody	372.6	4.5	-88
35.3539733886719	-80.4454727172852	circle1	green	2	Peabody	272.9	5.5	-87
35.3539276123047	-80.445556640625	circle1	green	3	Peabody	279.8	-0.8	-97
35.353931427002	-80.4455184936523	circle1	green	4	Peabody	276.2	-3.8	-97
35.3540534973145	-80.4456405639648	circle1	green	5	Peabody	291.4	-0.5	-94
35.3540420532227	-80.4456634521484	circle1	green	6	Peabody	293.3	-3.5	-99
35.3539085388184	-80.4457015991211	circle1	green	7	Peabody	293.5	4.0	-86
35.3536911010742	-80.4457855224609	circle1	green	8	Peabody	298.1	5.0	-90
35.3537101745605	-80.4457778930664	circle1	green	9	Peabody	297.6	6.2	-87
35.3537101745605	-80.4457626342773	circle1	green	10	Peabody	296.1	5.8	-87
35.3538055419922	-80.4464492797852	circle1	green	11	Peabody	365.2	-1.0	-94
35.3527374267578	-80.4468154907227	circle1	green